In [1]:
from datasets import fuzzy_boolean_dataset
import torch.nn as nn
from models import simple_machine, cls_machine, dini_machine
import torch
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import r2_score

In [22]:
[j for i in range(5) for j in 'ab']

['a', 'b', 'a', 'b', 'a', 'b', 'a', 'b', 'a', 'b']

In [27]:
x_size = 128
hid_size = 256
type_size = 300
num_hid_layer = 4 
type_inference_list = [nn.Linear(x_size, hid_size), nn.ReLU()] +\
        [sublayer for i in range(num_hid_layer) for sublayer in [nn.Linear(hid_size, hid_size), nn.ReLU()]] +\
        [nn.Linear(hid_size, type_size)]
m = nn.Sequential(*type_inference_list)
x = torch.randn((512, 128))
m(x).shape

torch.Size([512, 256])

In [21]:
batch_size = 128

In [22]:
dataset = fuzzy_boolean_dataset.FuzzyBooleanDataset('data/fbd.npy')
train_len = int( 0.8 * len(dataset) ) 
val_len = len(dataset) - train_len
train_set, val_set = torch.utils.data.random_split(dataset, [train_len, val_len])
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
    shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, \
    shuffle=True, num_workers=2)

In [23]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        #self.backbone = simple_machine.SimpleEncoder(32, 64, 2, 2, 4, 8)
        self.backbone = dini_machine.DiniEncoder(32, 8, 1, 4, 5, 8, 8, 0.01, 2, 1, 0.1)
        self.cls_net = cls_machine.ClsMachine(self.backbone, 30, 1)
        self.reg_head = nn.Linear(32, 1)
    
    def forward(self, x):
        temp = x 
        temp = self.cls_net(temp)
        temp = self.reg_head(temp)
        return temp

In [24]:
device = torch.device('cuda')
model = Model().to(device)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

35353

In [26]:
x = torch.randn((128, 10, 1)).to(device)
print(model(x).shape)
del x

torch.Size([128, 30, 1])


In [27]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

In [28]:
for epoch in range(20):
    running_loss = []
    pbar = tqdm(trainloader)
    for X, Y in pbar:
        X = X.to(device).unsqueeze(2).float()
        Y = Y.to(device).float()[:, :20]
        optimizer.zero_grad()
        pred = model(X).squeeze(2)[:, :20]
        loss = criterion(pred, Y)
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {epoch}")
        running_loss.append(loss.item())
        pbar.set_postfix(loss = sum(running_loss) / len(running_loss))
    
    with torch.no_grad():
        pbar = tqdm(valloader)
        scores = []
        for X, Y in pbar:
            X = X.to(device).unsqueeze(2).float().detach()
            Y = Y.to(device).float()[:, :20]
            pred = model(X).squeeze(2)[:, :20]
            score = r2_score(pred.detach().cpu(), Y.detach().cpu())
            scores.append(score)
            pbar.set_postfix(score = sum(scores) / len(scores))

100%|██████████| 256/256 [00:19<00:00, 12.84it/s, score=0.823]


In [ ]:
# backup = model
#model = backup
model_2 = model
optimizer = torch.optim.Adam([model_2.cls_net.cls_embeddings], lr=1e-3)

In [ ]:
for epoch in range(3):
    running_loss = []
    pbar = tqdm(trainloader)
    for X, Y in pbar:
        X = X.to(device).unsqueeze(2).float()
        Y = Y.to(device).float()[:, 20:]
        optimizer.zero_grad()
        pred = model_2(X).squeeze(2)[:, 20:]
        loss = criterion(pred, Y)
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {epoch}")
        running_loss.append(loss.item())
        pbar.set_postfix(loss = sum(running_loss) / len(running_loss))
    
    with torch.no_grad():
        pbar = tqdm(valloader)
        scores = []
        for X, Y in pbar:
            X = X.to(device).unsqueeze(2).float().detach()
            Y = Y.to(device).float()[:, 20:]
            pred = model_2(X).squeeze(2)[:, 20:]
            score = r2_score(pred.detach().cpu(), Y.detach().cpu())
            scores.append(score)
            pbar.set_postfix(score = sum(scores) / len(scores))

100%|██████████| 256/256 [00:19<00:00, 13.01it/s, score=-7.76]
